In [16]:
from keras.models import Sequential
from keras.layers import Dense

# import librart
import sklearn
import numpy as np
import pandas as pd
import pickle
from scipy.sparse import hstack
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import scipy
from scipy.sparse import csr_matrix
from sklearn.feature_selection import SelectKBest, mutual_info_classif
import feature_select
import feture_process
from scipy.sparse import load_npz
from scipy.sparse import load_npz
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [17]:
# use countvectorizer
# use countvectorizer
# use countvectorizer
# use countvectorizer

# import data
train_df = pd.read_csv("./project_data_files/book_rating_train.csv")
test_df = pd.read_csv("./project_data_files/book_rating_test.csv")

In [18]:
# process Prublisher and Language
train_df["Language"], test_df["Language"] = feture_process.docclass_preprocess(train_df["Language"],test_df["Language"],10)
train_df["Publisher"], test_df["Publisher"] = feture_process.docclass_preprocess(train_df["Publisher"],test_df["Publisher"],200)

In [19]:
# use oneHotCode for Publisher (sklearn)
publisher_train_hot = feture_process.process_OneHotEncoder(train_df,"Publisher")
publisher_test_hot = feture_process.process_OneHotEncoder(test_df,"Publisher")
train_df = pd.concat([train_df, publisher_train_hot], axis=1)
test_df = pd.concat([test_df, publisher_test_hot], axis=1)

In [20]:
# use oneHotCode for Language (sklearn)
language_train_hot = feture_process.process_OneHotEncoder(train_df,"Language")
language_test_hot = feture_process.process_OneHotEncoder(test_df,"Language")
train_df = pd.concat([train_df, language_train_hot], axis=1)
test_df = pd.concat([test_df, language_test_hot], axis=1)

In [21]:
# use oneHotCode for label (sklearn)
label_train_hot = feture_process.process_OneHotEncoder(train_df,"rating_label")
train_df = pd.concat([train_df, label_train_hot], axis=1)
train_df = train_df.drop(columns=["rating_label"])

In [22]:
# train_name_countvectorizer
train_name_countvectorizer = pickle.load(open("./project_data_files/book_text_features_countvec/train_name_countvectorizer.pkl", "rb"))
train_name_dic = train_name_countvectorizer.vocabulary_

# train_authors_countvectorizer
train_authors_countvectorizer = pickle.load(open("./project_data_files/book_text_features_countvec/train_authors_countvectorizer.pkl", "rb"))
train_authors_dic = train_authors_countvectorizer.vocabulary_

# train_desc_countvectorizer
train_desc_countvectorizer = pickle.load(open("./project_data_files/book_text_features_countvec/train_desc_countvectorizer.pkl", "rb"))
train_desc__dic = train_desc_countvectorizer.vocabulary_

# process vector features
train_name_features = train_name_countvectorizer.transform(train_df['Name'])
train_authors_features = train_authors_countvectorizer.transform(train_df['Authors'])
train_desc_features = train_desc_countvectorizer.transform(train_df['Description'])
other_features_df = train_df.drop(columns=['Name', 'Authors', 'Description', 'Publisher', 'Language'])

In [23]:
# new sparse features
sparse_features = hstack([train_name_features, train_authors_features, train_desc_features])
# new train features
dense_features = csr_matrix(other_features_df.values)
train_features = hstack([sparse_features, dense_features])


# process test features
test_name_features = scipy.sparse.load_npz('./project_data_files/book_text_features_countvec/test_name_vec.npz')
test_authors_features = scipy.sparse.load_npz('./project_data_files/book_text_features_countvec/test_authors_vec.npz')
test_desc_features = scipy.sparse.load_npz('./project_data_files/book_text_features_countvec/test_desc_vec.npz')
test_other_features_df = test_df.drop(columns=['Name', 'Authors', 'Description', 'Publisher', 'Language'])

test_sparse_features = hstack([test_name_features, test_authors_features, test_desc_features])
new_dense_features = csr_matrix(test_other_features_df.values)
test_features = hstack([test_sparse_features, new_dense_features])

In [ ]:
selected_features,selected_features_test = feature_select.MI(train_df,train_features,test_features,12000)

In [24]:
selected_features = load_npz("selected_features_label_one_hot_12000.npz")
selected_features_test = load_npz("selected_features_test_label_one_hot_12000.npz")

In [61]:
from keras.layers import Dropout
input_dim = selected_features.shape[1] 
num_classes = label_train_hot.shape[1]

model = Sequential()
model.add(Dense(40, activation='relu', input_shape=(input_dim,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [62]:
X_train, X_val, y_train, y_val = train_test_split(selected_features, label_train_hot, test_size=0.3, random_state=66)

In [63]:
X_train = X_train.toarray()
X_val = X_val.toarray()

In [64]:
mlp = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/50
505/505 [==============================] - 2s 4ms/step - loss: 0.9357 - accuracy: 0.6797 - val_loss: 0.7094 - val_accuracy: 0.7211
Epoch 2/50
505/505 [==============================] - 2s 3ms/step - loss: 0.6488 - accuracy: 0.7228 - val_loss: 0.7375 - val_accuracy: 0.6774
Epoch 3/50
505/505 [==============================] - 2s 3ms/step - loss: 0.5920 - accuracy: 0.7449 - val_loss: 0.6910 - val_accuracy: 0.7260
Epoch 4/50
505/505 [==============================] - 2s 3ms/step - loss: 0.5480 - accuracy: 0.7641 - val_loss: 0.6858 - val_accuracy: 0.7057
Epoch 5/50
505/505 [==============================] - 2s 3ms/step - loss: 0.5066 - accuracy: 0.7862 - val_loss: 0.6883 - val_accuracy: 0.6932
Epoch 6/50
505/505 [==============================] - 2s 3ms/step - loss: 0.4683 - accuracy: 0.7986 - val_loss: 0.6778 - val_accuracy: 0.7252
Epoch 7/50
505/505 [==============================] - 2s 3ms/step - loss: 0.4271 - accuracy: 0.8193 - val_loss: 0.6984 - val_accuracy: 0.7403
Epoch 

In [ ]:
prediction_prob = model.predict(selected_features_test)

In [ ]:
# get predictions
prediction_index = np.argmax(prediction_prob, axis=1)
ratings = np.array([3.0, 4.0, 5.0])
prediction = ratings[prediction_index]

In [ ]:
output_df = pd.DataFrame({'rating_label': prediction})
output_df.index += 1
output_df.index.name = 'id'
output_df.to_csv('./predictions.csv')